In [2]:
import mido as md
import numpy

In [9]:
mid = md.MidiFile('data/JazzMidi/55Dive.mid')

TypeError: object of type 'MidiFile' has no len()

In [14]:
t = n = 0
for i, track in enumerate(mid.tracks):
    t += 1
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(msg)
        n += 1
    print(n)
print(t)


Track 0: untitled
MetaMessage('track_name', name='untitled', time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('key_signature', key='Eb', time=0)
MetaMessage('set_tempo', tempo=600000, time=0)
MetaMessage('marker', text='Intro', time=480)
MetaMessage('marker', text='A1', time=3840)
MetaMessage('time_signature', numerator=2, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=1920)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=240)
MetaMessage('time_signature', numerator=2, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=2880)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=240)
MetaMessage('key_signature', key='C', time=0)
MetaMessage('marker', text='A2', time=0)
MetaMessage('time_signature', numerator=2, denominator=4, cl

note_on channel=1 note=66 velocity=68 time=4237
note_on channel=1 note=66 velocity=0 time=23
note_on channel=1 note=63 velocity=68 time=0
note_on channel=1 note=63 velocity=0 time=20
note_on channel=1 note=60 velocity=68 time=0
note_on channel=1 note=60 velocity=0 time=20
note_on channel=1 note=55 velocity=68 time=0
note_on channel=1 note=55 velocity=0 time=16
note_on channel=1 note=59 velocity=68 time=4
note_on channel=1 note=59 velocity=0 time=40
note_on channel=1 note=60 velocity=68 time=0
note_on channel=1 note=60 velocity=0 time=24
note_on channel=1 note=63 velocity=68 time=56
note_on channel=1 note=63 velocity=0 time=40
note_on channel=1 note=57 velocity=68 time=0
note_on channel=1 note=57 velocity=0 time=24
note_on channel=1 note=65 velocity=68 time=96
note_on channel=1 note=63 velocity=68 time=20
note_on channel=1 note=65 velocity=0 time=4
note_on channel=1 note=63 velocity=0 time=20
note_on channel=1 note=62 velocity=68 time=76
note_on channel=1 note=71 velocity=68 time=20
not

In [31]:
print(type(enumerate(mid.tracks)))

<class 'enumerate'>
